
# Convolutional Neural Network For Image Classification

- CNN이란?
- Convolutional Neural Network For Image Classification
  - Approach 1 :
  - Approach 2 :

## CNN(Convolutional Neural Network)
- CNN은 ANN 이전에 Convolution 연산을 통해 이미지의 특징을 자동으로 찾는 기능을 추가한 것이다.

- CNN의 구성은 합성곱 연산 -> 풀링 -> 평탄화 -> 밀집 신경망 으로 기본적으로 구성된다.

### Importing the Libraries & Loading the image

- tensorflow로 학습을 하기위해선 training_set과 test_set으로 폴더를 나누어서 관리해야된다.

In [13]:
import cv2
import numpy as np
import os
import zipfile
from google.colab.patches import cv_imshow
import tensorflow as tf
from tensorflow .keras.models import Sequential
# Convolution 연산을 위한 import
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
# 이미지 전처리를 도와주는 객체( 테스트 데이터, 검증 데이터 분할, 이미지 늘리기 등 )
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import seaborn as sns # 데이터 시각화
import matplotlib.pyplot as plt # 그래프 생성
tf.__version__

'2.15.0'

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
path = '/content/drive/MyDrive/Colab Notebooks/Project/Computer Vision/src/Datasets/homer_bart_2.zip'
zip_object = zipfile.ZipFile(file= path, mode = 'r')
zip_object.extractall('./')
zip_object.close()

In [16]:
# tf를 사용하여 이미지 loading 방법
# tf.keras.preprocessing.image.load_img('')

### Train and Test set
- tensorflow의 ImageDataGenerator 객체를 활용하여 데이터 전처리
  - rescale : rescale 값은 이미지의 픽셀 값을 0과 1 사이의 값 정규화 하는데 사용한다. 예를 들어, rescale=1./255는 이미지의 픽셀 값을 0과 1 사이의 값으로 조정하고 rescale=2./255 - 1 는 이미지를 -1과 1 사이의 값으로 조정한다. "1./255"에서 1.은 1.0(소수점 표현)이고 /는 나눗셈 연산 255는 나눌 값이다(색상의 값 범위)
  - 이미지 증강 설정: 이미지 데이터가 적을때 전처리를 통해 이미지 데이터를 늘릴 수 있다. 다음과 같은 매개변수를 활용하라
    - horizontal_flip : 이미지 수평 뒤집기
    - rotation_range : 이미지 회전 정도
    - zoom_range=0.2 : 이미지 확대 및 축소

In [17]:
training_generator = ImageDataGenerator(rescale= 1./255, rotation_range=7, horizontal_flip =True, zoom_range=0.2)

In [18]:
# flow_from_directory : 실제 데이터 로드, shuffle = true 한 이유는 순서가 일정하면 특징이 아닌 순서를 통한 학습을 하기때문
train_dataset = training_generator.flow_from_directory('/content/homer_bart_2/training_set', target_size = (64, 64), batch_size =8, class_mode ='categorical', shuffle = True)

Found 215 images belonging to 2 classes.


In [19]:
train_dataset.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [20]:
train_dataset.class_indices

{'bart': 0, 'homer': 1}

In [21]:
test_generator = ImageDataGenerator(1./255)
test_dataset = test_generator.flow_from_directory('/content/homer_bart_2/test_set', target_size = (64, 64), batch_size =1, class_mode = 'categorical', shuffle = False)

Found 54 images belonging to 2 classes.


# Building and training the neural network

In [25]:
network = Sequential()
# conv2D : convolutional층을 생성
# 매개변수 : filters = 커널의 개수, kernel_size = 커널의 크기, activation = 특징맵에 적용할 활성화 함수
network.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = (64,64,3)))
# conv층에서 생성된 특징맵을 폴링을 이용하여 특징을 확실한 특징 필터링
network.add(MaxPool2D(pool_size=(2,2)))

network.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
network.add(MaxPool2D(pool_size=(2,2)))

network.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
network.add(MaxPool2D(pool_size=(2,2)))

# 행렬형태의 특징맵을 백터화(일렬로 세움)
network.add(Flatten())

network.add(Dense(units = 577, activation ='relu'))

network.add(Dense(units = 577, activation ='relu'))
# softmax는 sigmoid와 달리 이진분류가 아닌 다분류 문제에서 사
network.add(Dense(units = 2, activation ='softmax'))

# 요약을 활용하여 은닉층의 뉴런 개수를 정하자
# (1152+2)/2 =577
network.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 31, 31, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 12, 12, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 6, 6, 32)         

In [26]:
# loss 함수는 이진분류(binary)이면 binary_crossentropy를 사용, 다분류(categorical)이면 해당 함수를 사용한다.
network.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics =['accuracy'])

In [27]:
history = network.fit_generator(train_dataset, epochs=50)

Epoch 1/50


<ipython-input-27-5df4b5f3c6a2>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = network.fit_generator(train_dataset, epochs=50)


27/27 [==============================] - 3s 65ms/step - loss: 0.7208 - accuracy: 0.5767
Epoch 2/50
27/27 [==============================] - 3s 92ms/step - loss: 0.6680 - accuracy: 0.5814
Epoch 3/50
27/27 [==============================] - 2s 56ms/step - loss: 0.5966 - accuracy: 0.6837
Epoch 4/50
27/27 [==============================] - 2s 56ms/step - loss: 0.5533 - accuracy: 0.7488
Epoch 5/50
27/27 [==============================] - 2s 56ms/step - loss: 0.4979 - accuracy: 0.7488
Epoch 6/50
27/27 [==============================] - 2s 55ms/step - loss: 0.4441 - accuracy: 0.7674
Epoch 7/50
27/27 [==============================] - 3s 91ms/step - loss: 0.4092 - accuracy: 0.8093
Epoch 8/50
27/27 [==============================] - 2s 55ms/step - loss: 0.4403 - accuracy: 0.8186
Epoch 9/50
27/27 [==============================] - 2s 56ms/step - loss: 0.3808 - accuracy: 0.8233
Epoch 10/50
27/27 [==============================] - 2s 55ms/step - loss: 0.3202 - accuracy: 0.8512
Epoch 11/50
27/27 [=